<h1>День 2. Модели склонности</h1>

<h3>Сервисный модуль</h3>
<b><i>Пожалуйста, запустите следующие несколько ячеек при первом запуске ноутбука!</i></b>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%pip install -r '/home/jupyter/work/resources/a2nced/requirements.txt'

In [ ]:
import os
import sys
from pathlib import Path
dir_path = Path(os.getcwd()).resolve().parent.as_posix()
sys.path.append(dir_path)

<h3>Конец сервисного модуля</h3>

<br>
<h1>Практика 1</h1>
<h3>Построение базовой модели склонности</h3>
<br>
<h4>Подготовка данных</h4>

In [ ]:
# необходимые импорты
import pandas as pd
import numpy as np
import datetime as dt

In [ ]:
# необходимые импорты
import sklearn
from lightgbm import LGBMClassifier

In [ ]:
# найдем директорию, в которой хранятся наши данные
_PATH = '/home/jupyter/mnt/s3/course-test/day_2/'

In [ ]:
# укажем пути до конкретных файлов
transactions_path = os.path.join(_PATH, 'day_2_transactions.csv') # транзакции
demographics_path = os.path.join(_PATH, 'day_2_demogr.csv') # информация о клиентах
prod_path         = os.path.join(_PATH, 'day_2_product.csv') # информация о товарах
orders_path       = os.path.join(_PATH, 'day_2_orders.csv') # информация о заказах, сгруппированная по корзинам

In [ ]:
# загрузим данные
transactions_df = pd.read_csv(transactions_path)
transactions_df['purch_date_time'] = transactions_df['purch_date_time'].astype(np.datetime64)

In [ ]:
demographics = pd.read_csv(demographics_path)
prod_df = pd.read_csv(prod_path)

In [ ]:
# загрузим данные по заказам, которые мы собрали на прошлом занятии
orders_df = pd.read_csv(orders_path)
orders_df['purch_date_time'] = orders_df['purch_date_time'].astype(np.datetime64)
orders_df.head()

In [ ]:
available_ids = demographics['user_id'].values

<br>
<h4>Автоматическая генерация признаков</h4>

In [ ]:
# Перед началом работы необходимо описать наши данные и связи между ними

In [ ]:

import featuretools as ft
ft.__version__

In [ ]:
# для удобства оформим повторно используемый код в виде функции
def create_feature_matrix(entity_set:ft.entityset.entityset.EntitySet,
                          cutoff:str,
                          common_cols:list=['user_id', 'age_desc', 'income_desc',
                                            'have_own_house', 'num_adults',
                                            'adult_gender', 'no_kids', 'size']
                         ) -> pd.DataFrame:
    feature_matrix_14, feature_defs_14 = ft.dfs(entityset=entity_set,
                                          target_dataframe_name='customer_info',
                                          cutoff_time=cutoff,
                                          training_window="14 days",
                                          verbose=True)
    feature_matrix_30, feature_defs_30 = ft.dfs(entityset=entity_set,
                                          target_dataframe_name='customer_info',
                                          cutoff_time=cutoff,
                                          training_window="30 days",
                                          verbose=True)
    feature_matrix_60, feature_defs_60 = ft.dfs(entityset=entity_set,
                                          target_dataframe_name='customer_info',
                                          cutoff_time=cutoff,
                                          training_window="60 days",
                                          verbose=True)
    
    feature_matrix_14 = feature_matrix_14.reset_index()
    feature_matrix_30 = feature_matrix_30.reset_index()
    feature_matrix_60 = feature_matrix_60.reset_index()
    feature_matrix_14.columns = feature_matrix_14.columns.map(lambda x: f'{x}_14_days' if x not in common_cols else str(x))
    feature_matrix_30.columns = feature_matrix_30.columns.map(lambda x: f'{x}_30_days' if x not in common_cols else str(x))
    feature_matrix_60.columns = feature_matrix_60.columns.map(lambda x: f'{x}_60_days' if x not in common_cols else str(x))
    feature_matrix = feature_matrix_14.merge(feature_matrix_30, how='left')
    feature_matrix = feature_matrix.merge(feature_matrix_60, how='left')
    return(feature_matrix)

In [ ]:
# теперь создадим описание нашей модели данных для создания витрин

In [ ]:
# чтобы получить агрегаты, мы должны указать объекты и связи между ними
# для этого создаем отдельную сущность
es = ft.EntitySet("a2nced_sample_2")

In [ ]:
# создадим уникальное поле для каждой позиции в каждом заказе transactions_df
transactions_df['record_id'] = transactions_df.index

In [ ]:
# обогатим наши данные информацией о продуктах
trans_catig_df = transactions_df.merge(prod_df[['product_id', 'manufacturer',
                                                'department', 'brand']],
                                       how='left',
                                       on='product_id')
trans_catig_df.head()

In [ ]:
# укажем логические типы данных, которые не следуют явно из типов хранимых данных:
# иначе он будет вычислять среднее значение id производителя
customer_info_types = {
    'user_id': "Categorical",
    'age_desc': "Categorical",
    'income_desc': "Categorical",
    'have_own_house': "BooleanNullable",
    'num_adults': "Categorical",
    'adult_gender': "Categorical",
    'no_kids': "BooleanNullable",
    'size': "Categorical"
}

products_types = {
    'product_id': "Categorical",
    'manufacturer': "Categorical",
    'department': "Categorical",
    'brand': "Categorical",
    'commodity_desc': "Categorical",
    'sub_commodity_desc': "Categorical",
    'curr_size_of_product': "Categorical"
}

orders_types = {
    'user_id': "Categorical",
    'basket_id': "Categorical",
    'purch_date_time': "Datetime",
    'quantity': "Double",
    'sales_value': "Double",
    'retail_disc': "Double",
    'default_price': "Double"    
}

transactions_types ={
    'user_id': "Categorical",
    'purch_date_time': "Datetime",
    'basket_id': "Categorical",
    'product_id': "Categorical",
    'quantity': "Double",
    'sales_value': "Double",
    'retail_disc': "Double",
    'default_price': "Double",
    'record_id': "Categorical",
    'manufacturer': "Categorical",
    'department': "Categorical",
    'brand': "Categorical",
}

In [ ]:
# сначала добавим созданные датафреймы
es.add_dataframe(dataframe_name='customer_info',
                 dataframe=demographics,
                 logical_types=customer_info_types,
                 index='user_id')

es.add_dataframe(dataframe_name='orders',
                 dataframe=orders_df,
                 logical_types=orders_types,
                 index='basket_id',
                 time_index='purch_date_time')

es.add_dataframe(dataframe_name='transactions',
                 dataframe=trans_catig_df.loc[trans_catig_df.user_id.isin(available_ids)],
                 logical_types=transactions_types,
                 index='record_id',
                 time_index='purch_date_time')

es.add_dataframe(dataframe_name='products',
                 dataframe=prod_df,
                 logical_types=products_types,
                 index='product_id')

In [ ]:
es.add_relationship('customer_info', 'user_id', 'orders', 'user_id')
es.add_relationship('customer_info', 'user_id', 'transactions', 'user_id')
es.add_relationship('orders', 'basket_id', 'transactions', 'basket_id')
es.add_relationship('products', 'product_id', 'transactions', 'product_id')

<h4>Сбор обучающей выборки</h4>

In [ ]:
# получим нашу витрину с агрегатами за 14, 30 и 60 дней с помощью функции, которую мы описали ранее

feature_matrix = create_feature_matrix(es, '2018-10-08') # нужно указать источник данных и дату среза

<h4>Разметка целевого события</h4>

In [ ]:
# соберем сведения о наиболее популярных продуктах
# важно использовать именно count(), иначе мы получим информацию по заправке, а не по собственно, магазину
top_10_prods = transactions_df.groupby('product_id')['quantity'].count().reset_index().sort_values('quantity', ascending=False).head(10)

In [ ]:
prod_id = top_10_prods.product_id.values[0]

In [ ]:
start_period = dt.datetime.strptime('2019-07-01', '%Y-%m-%d') + dt.timedelta(days=1)
end_period =  dt.datetime.strptime('2019-07-01', '%Y-%m-%d')  + dt.timedelta(days=14)

In [ ]:
# соберем_таргет
target_mask = ((transactions_df['user_id'].isin(available_ids)) &\
              (transactions_df['purch_date_time'] >= start_period) &\
              (transactions_df['purch_date_time'] <= end_period) &\
              (transactions_df['product_id'] == prod_id))

target = pd.DataFrame()
target['user_id'] = transactions_df.loc[target_mask]['user_id'].unique()
target['product_id'] = prod_id

# маркируем целевое действие - совершение покупки
target['y'] = 1
target.head()

In [ ]:
df_one_prod = feature_matrix.merge(target, how='left')

In [ ]:
df_one_prod['y'] = df_one_prod['y'].fillna(0).astype('int8')

In [ ]:
X = df_one_prod.drop(['user_id', 'product_id', 'y'], axis=1)
y = df_one_prod.y

In [ ]:
y.value_counts()

In [ ]:
# lightgbm не дружит с типом данных 'category'
for column in X.select_dtypes(exclude=['Int64', 'float64']).columns.values:
    X[column] = X[column].astype('object')

    
X = X.replace({'<NA>': np.nan})
X = X.replace({True: 1, False: 0})
X = X.replace({pd.NA: np.nan})

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [ ]:
categorical_features = X.select_dtypes(exclude=['Int64', 'float64']).columns.values
numeric_features = [feature for feature in list(X.columns) if feature not in categorical_features]

<h4>Подбор оптимальных гиперпараметров и обучение модели</h4>

In [ ]:
from utils.models import create_pipeline, param_search, search_space, unpack_model_params

In [ ]:
pipeline = create_pipeline(cat_features=categorical_features,
                          numer_features=numeric_features,
                          model=LGBMClassifier(objective = "binary", random_state = 42, is_unbalance=True))

In [ ]:
best = param_search(pipeline, X_train, y_train, search_space)

In [ ]:
best_params = unpack_model_params(best)
pipeline.set_params(**best_params)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
prob_pred = pipeline.predict_proba(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, plot_roc_curve
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
plot_roc_curve(pipeline, X_test, y_test)

<h4>Оценка качества модели на отложенной выборке (OOT - out of time)</h4>

In [ ]:
feature_matrix_oot = create_feature_matrix(es, '2019-07-15')

In [ ]:
start_period_oot = dt.datetime.strptime('2019-07-15', '%Y-%m-%d') + dt.timedelta(days=1)
end_period_oot = dt.datetime.strptime('2019-07-15', '%Y-%m-%d') + dt.timedelta(days=14)

In [ ]:
# соберем таргет
target_mask = ((transactions_df['user_id'].isin(available_ids)) &\
              (transactions_df['purch_date_time'] >= start_period) &\
              (transactions_df['purch_date_time'] <= end_period) &\
              (transactions_df['product_id'] == prod_id))

target_oot = pd.DataFrame()
target_oot['user_id'] = transactions_df.loc[target_mask]['user_id'].unique()
target_oot['product_id'] = prod_id

# маркируем целевое действие - совершение покупки
target_oot['y'] = 1
target_oot.head()

In [ ]:
df_prod_oot = feature_matrix_oot.merge(target_oot, how='left')

In [ ]:
df_prod_oot.info()

In [ ]:
df_prod_oot['y'] = df_prod_oot['y'].fillna(0).astype('int8')

In [ ]:
X_oot = df_prod_oot.drop(['user_id', 'product_id', 'y'], axis=1)
y_oot = df_prod_oot.y

In [ ]:
y_pred_oot = pipeline.predict(X_oot)
y_pred_prob_oot = pipeline.predict_proba(X_oot)

In [ ]:
print(f'Accuracy: {accuracy_score(y_oot, y_pred_oot)}')

In [ ]:
plot_roc_curve(pipeline, X_oot, y_oot)

In [ ]:
scores = y_pred_prob_oot
scores = pd.DataFrame(scores)

scores['y'] = y_oot.values
scores_sorted = scores.sort_values(1, ascending=False)[[1, 'y']]
scores_sorted['cum_recall'] = scores_sorted['y'].cumsum()
scores_sorted['position'] = range(1, scores_sorted.shape[0]+1)
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]+1

scores.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_diff'] = scores_sorted['recall_at'].diff()
scores_sorted['position_diff'] = scores_sorted['adj_position'].diff()

scores_sorted.fillna(0, inplace=True)

scores_sorted['derivative'] = scores_sorted['position_diff']  / (scores_sorted['recall_diff']+1e-6)
scores_sorted['sum_derivative'] = scores_sorted['derivative'].cumsum()

scores_sorted.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]

In [ ]:
scores_sorted.plot(x='adj_position', y='recall_at', figsize=(12,8), title='Кумулятивный recall')

In [ ]:
from utils.metrics import find_inflection_point
cutoff = find_inflection_point(scores_sorted, 5, 4, X.shape[0])

In [ ]:
cutoff

In [ ]:
cutoff = cutoff / scores_sorted.shape[0]

In [ ]:
import pickle

_FILEPATH = os.path.join(os.path.dirname(os.getcwd()), 'references', 'models', 'day_2_base_prop.pkl')

with open(_FILEPATH, 'wb') as pickle_file:
    pickle.dump(pipeline, pickle_file)

<h4>Формирование сегмента клиентов для пилота</h4>

In [ ]:
feature_matrix_score = create_feature_matrix(es, '2019-07-29')

In [ ]:
X_score = feature_matrix_score.drop('user_id', axis=1)

In [ ]:
scores = pipeline.predict_proba(X_score)

In [ ]:
score_df = pd.DataFrame()
score_df['customer_id'] = feature_matrix_score.user_id
score_df['report_date'] = dt.date.today()
score_df['score'] = scores[:,1]

In [ ]:
score_df.sort_values('score', ascending=False).iloc[:int(cutoff * score_df.shape[0]) +1, 0].values

<h3>Домашнее задание 1</h3>
<br>
Для полученной выборки разметьте таргет<br>
Целевое событие: покупка клиентом другого популярного товара.<br>
<b><i>Подсказка:</i></b> Используйте любой другой товар из собранной нами топ-10 популярных товаров.

<br>
<h1>Практика 2</h1>
<h3>Углубленное целевое событие</h3>
<br>
<h4>Сбр обучающей выборки</h4>

In [ ]:
# таргет - две покупки в выбранной категории

In [ ]:
df = feature_matrix.copy()

In [ ]:
department_name = 'PASTRY'
start_period = dt.datetime.strptime('2019-07-01', '%Y-%m-%d') + dt.timedelta(days=1)
end_period = dt.datetime.strptime('2019-07-01', '%Y-%m-%d') + dt.timedelta(days=21)

<h4>Разметка целевого события</h4>

In [ ]:
mask_target = (trans_catig_df['purch_date_time'] >= start_period) &\
                (trans_catig_df['purch_date_time'] <= end_period) &\
                (trans_catig_df['department'] == department_name) &\
                (trans_catig_df.user_id.isin(available_ids))

In [ ]:
target = trans_catig_df.loc[mask_target].groupby('user_id')['basket_id'].nunique().reset_index()

In [ ]:
target['y'] = target['basket_id'].apply(lambda x: 1 if x >= 2 else 0)
target = target[['user_id', 'y']]
target['y'].value_counts()

<h3>Домашнее задание 2</h3>
<br>
Для полученной выборки разметьте таргет<br>
Целевое событие: покупка клиентом определенного товара два или более раз в период двух недель от даты витрины.<br>
<b><i>Подсказка:</i></b> Можете использовать любой id продукта.

<h3>Домашнее задание 3</h3>
<br>
Для полученной выборки разметьте таргет<br>
Целевое событие: покупка клиентом определенного товара в период двух недель от даты витрины или три и более покупки этого же товара в период четырех недель от даты витрины.<br>
<b><i>Подсказка:</i></b> Покупки в период четырех недель от даты витрины включают и покупку в период двух недель от даты витрины.

<h3>Домашнее задание 4</h3>
<br>
Для полученной выборки разметьте таргет<br>
Целевое событие: покупка клиентом определенного товара в период одной недели от даты витрины и покупка того же товара в период третьей-четвертой недель от даты витрины.<br>
<b><i>Подсказка:</i></b> Попробуйте применить подход на основе множеств.

In [ ]:
df = df.merge(target, how='left', on='user_id')
df['y'] = df['y'].fillna(0)
df['y'] = df['y'].astype('int8')

In [ ]:
X = df.drop(columns=['user_id', 'y'], axis=1)
y = df['y']

In [ ]:
y.value_counts()

In [ ]:
# lightgbm не дружит с типом данных 'category'
for column in X.select_dtypes(exclude=['Int64', 'float64']).columns.values:
    X[column] = X[column].astype('object')

    
X = X.replace({'<NA>': np.nan})
X = X.replace({True: 1, False: 0})
X = X.replace({pd.NA: np.nan})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [ ]:
categorical_features = X.select_dtypes(exclude=['Int64', 'float64']).columns.values
numeric_features = [feature for feature in list(X.columns) if feature not in categorical_features]

<h4>Подбор оптимальных гиперпараметров и обучение модели</h4>

In [ ]:
pipeline = create_pipeline(cat_features=categorical_features,
                          numer_features=numeric_features,
                          model=LGBMClassifier(objective = "binary", random_state = 42, is_unbalance=True))

In [ ]:
best = param_search(pipeline, X_train, y_train, search_space)

In [ ]:
best_params = unpack_model_params(best)
pipeline.set_params(**best_params)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
prob_pred = pipeline.predict_proba(X_test)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
plot_roc_curve(pipeline, X_test, y_test)

<h4>Оценка качества модели на отложенной выборке (OOT - out of time)</h4>

In [ ]:
start_period = dt.datetime.strptime('2019-07-15', '%Y-%m-%d') + dt.timedelta(days=1)
end_period = dt.datetime.strptime('2019-07-15', '%Y-%m-%d') + dt.timedelta(days=21)

mask_target_oot = (trans_catig_df['purch_date_time'] >= start_period) &\
                (trans_catig_df['purch_date_time'] <= end_period) &\
                (trans_catig_df['department'] == department_name) &\
                (trans_catig_df.user_id.isin(available_ids))


target_oot = trans_catig_df.loc[mask_target_oot].groupby('user_id')['basket_id'].nunique().reset_index()


target_oot['y'] = target_oot['basket_id'].apply(lambda x: 1 if x >= 2 else 0)
target_oot = target_oot[['user_id', 'y']]
target_oot['y'].value_counts()

df_oot = feature_matrix_oot.copy()

df_oot = df_oot.merge(target_oot, how='left', on='user_id')
df_oot['y'] = df_oot['y'].fillna(0)
df_oot['y'] = df_oot['y'].astype('int8')
df_oot.y.value_counts()

In [ ]:
y_oot = df_oot['y']
X_oot = df_oot.drop(columns=['user_id', 'y'], axis=1)

In [ ]:
y_pred_oot = pipeline.predict(X_oot)
y_pred_prob_oot = pipeline.predict_proba(X_oot)

In [ ]:
print(f'Accuracy: {accuracy_score(y_oot, y_pred_oot)}')

In [ ]:
plot_roc_curve(pipeline, X_oot, y_oot)

In [ ]:
scores = y_pred_prob_oot
scores = pd.DataFrame(scores)

scores['y'] = y_oot.values
scores_sorted = scores.sort_values(1, ascending=False)[[1, 'y']]
scores_sorted['cum_recall'] = scores_sorted['y'].cumsum()
scores_sorted['position'] = range(1, scores_sorted.shape[0]+1)
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]+1

scores.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_diff'] = scores_sorted['recall_at'].diff()
scores_sorted['position_diff'] = scores_sorted['adj_position'].diff()

scores_sorted.fillna(0, inplace=True)

scores_sorted['derivative'] = scores_sorted['position_diff']  / (scores_sorted['recall_diff']+1e-6)
scores_sorted['sum_derivative'] = scores_sorted['derivative'].cumsum()

scores_sorted.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]

In [ ]:
scores_sorted.plot(x='adj_position', y='recall_at', figsize=(12,8), title='Кумулятивный recall')

In [ ]:
cutoff = find_inflection_point(scores_sorted, 5, 4, 801)

In [ ]:
cutoff

In [ ]:
cutoff = cutoff / scores_sorted.shape[0]

In [ ]:
_FILEPATH = os.path.join(os.path.dirname(os.getcwd()), 'references', 'models', 'day_2_comp_target.pkl')

with open(_FILEPATH, 'wb') as pickle_file:
    pickle.dump(pipeline, pickle_file)

<h4>Формирование сегмента клиентов для пилота</h4>

In [ ]:
X_score = feature_matrix_score.drop('user_id', axis=1)

In [ ]:
scores = pipeline.predict_proba(X_score)

In [ ]:
score_df = pd.DataFrame()
score_df['customer_id'] = feature_matrix_score.user_id
score_df['report_date'] = dt.date.today()
score_df['score'] = scores[:,1]

In [ ]:
score_df.sort_values('score', ascending=False).iloc[:int(cutoff * score_df.shape[0]) +1, 0].values

<br>
<h1>Практика 3</h1>
<h3>Построение единой модели склонности для нескольких продуктов</h3>

<h4>Сбор обучающей выборки</h4>

In [ ]:
# подготовим датафрейм, который будет содержать признаки продукта
prod_features = pd.DataFrame()
for prod_id in top_10_prods.product_id.values:
    temp_df = pd.DataFrame()
    temp_df['user_id'] = available_ids
    temp_df['product_id'] = prod_id
    prod_features = pd.concat([prod_features, temp_df], axis=0)

In [ ]:
def get_targets(start_period, end_period):
    targets = pd.DataFrame()
    for prod_id in top_10_prods.product_id.values:
        target_mask = ((transactions_df['user_id'].isin(available_ids)) &\
                      (transactions_df['purch_date_time'] >= start_period) &\
                      (transactions_df['purch_date_time'] <= end_period) &\
                      (transactions_df['product_id'] == prod_id))

        target = pd.DataFrame()
        target['user_id'] = transactions_df.loc[target_mask]['user_id'].unique()
        target['product_id'] = prod_id

        # маркируем целевое действие - совершение покупки
        target['y'] = 1
        targets = pd.concat([targets, target], axis=0)
    return(targets)

In [ ]:
start_period = dt.datetime.strptime('2019-07-01', '%Y-%m-%d') + dt.timedelta(days=1)
end_period = dt.datetime.strptime('2019-07-01', '%Y-%m-%d') + dt.timedelta(days=21)
targets = get_targets(start_period, end_period)

In [ ]:
df_prod = prod_features.merge(feature_matrix, how='left')
df_prod = df_prod.merge(targets, how='left', on=['user_id', 'product_id'])

In [ ]:
df_prod['y'] = df_prod['y'].fillna(0).astype('int8')

In [ ]:
df_prod = df_prod.sample(2000)

In [ ]:
X = df_prod.drop(['user_id', 'y'], axis=1)
y = df_prod.y

In [ ]:
# lightgbm не дружит с типом данных 'category'
for column in X.select_dtypes(exclude=['Int64', 'float64']).columns.values:
    X[column] = X[column].astype('object')

    
X = X.replace({'<NA>': np.nan})
X = X.replace({True: 1, False: 0})
X = X.replace({pd.NA: np.nan})

In [ ]:
y.value_counts()

In [ ]:
categorical_features = X_train.select_dtypes(exclude=['Int64', 'float64']).columns.values
categorical_features = np.append(categorical_features, 'product_id')
numeric_features = [feature for feature in list(X.columns) if feature not in categorical_features]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [ ]:
pipeline = create_pipeline(cat_features=categorical_features,
                          numer_features=numeric_features,
                          model=LGBMClassifier(objective = "binary", random_state = 42, is_unbalance=True))

In [ ]:
# снизим количество эпох, чтобы емного ускорить обучение
best = param_search(pipeline, X_train, y_train, search_space, 50)

In [ ]:
best_params = unpack_model_params(best)
pipeline.set_params(**best_params)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
prob_pred = pipeline.predict_proba(X_test)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
plot_roc_curve(pipeline, X_test, y_test)

<h3>Калибровка</h3>

In [ ]:
# Схожю по функционалу модель можно построить на основе моделей отлклика для одного продукта на основе калибровки

from sklearn.calibration import CalibratedClassifierCV


# загрузим предобученную ранее модель
_FILEPATH = os.path.join(os.path.dirname(os.getcwd()), 'references', 'models', 'day_2_base_prop.pkl')

with open(_FILEPATH, 'rb') as pickle_file:
    fitted_model = pickle.load(pickle_file)

calibrated_model = CalibratedClassifierCV(base_estimator=fitted_model, cv='prefit', method='isotonic')
X_train_calib = X_train.drop(['product_id'], axis=1)
calibrated_model.fit(X, y)

In [ ]:
X_oot

In [ ]:
y_pred_oot = calibrated_model.predict(X_oot)
y_pred_prob_oot = calibrated_model.predict_proba(X_oot)

In [ ]:
print(f'Accuracy: {accuracy_score(y_oot, y_pred_oot)}')

In [ ]:
plot_roc_curve(calibrated_model, X_oot, y_oot)

<h4>Оценка качества модели на отложенной выборке (OOT - out of time)</h4>

In [ ]:
start_period_oot = dt.datetime.strptime('2019-07-15', '%Y-%m-%d') + dt.timedelta(days=1)
end_period_oot = dt.datetime.strptime('2019-07-15', '%Y-%m-%d') + dt.timedelta(days=21)

In [ ]:
target_oot = get_targets(start_period_oot, end_period_oot)

In [ ]:
df_prod_oot = prod_features.merge(feature_matrix_oot, how='left')
df_prod_oot = df_prod_oot.merge(target_oot, how='left')

In [ ]:
df_prod_oot['y'] = df_prod_oot['y'].fillna(0).astype('int8')

In [ ]:
X_oot = df_prod_oot.drop(['user_id', 'y'], axis=1)
y_oot = df_prod_oot.y

In [ ]:
y_oot.value_counts()

In [ ]:
y_pred_oot = pipeline.predict(X_oot)
y_pred_prob_oot = pipeline.predict_proba(X_oot)

In [ ]:
print(f'Accuracy: {accuracy_score(y_oot, y_pred_oot)}')

In [ ]:
plot_roc_curve(pipeline, X_oot, y_oot)

In [ ]:
scores = y_pred_prob_oot
scores = pd.DataFrame(scores)

scores['y'] = y_oot.values
scores_sorted = scores.sort_values(1, ascending=False)[[1, 'y']]
scores_sorted['cum_recall'] = scores_sorted['y'].cumsum()
scores_sorted['position'] = range(1, scores_sorted.shape[0]+1)
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]+1

scores.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_diff'] = scores_sorted['recall_at'].diff()
scores_sorted['position_diff'] = scores_sorted['adj_position'].diff()

scores_sorted.fillna(0, inplace=True)

scores_sorted['derivative'] = scores_sorted['position_diff']  / (scores_sorted['recall_diff']+1e-6)
scores_sorted['sum_derivative'] = scores_sorted['derivative'].cumsum()

scores_sorted.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]

In [ ]:
scores_sorted.plot(x='adj_position', y='recall_at', figsize=(12,8), title='Кумулятивный recall')

In [ ]:
cutoff = find_inflection_point(scores_sorted, 5, 4, X.shape[0])

In [ ]:
cutoff

In [ ]:
cutoff = cutoff / scores_sorted.shape[0]

In [ ]:
_FILEPATH = os.path.join(os.path.dirname(os.getcwd()), 'references', 'models', 'day_2_multi_prod.pkl')

with open(_FILEPATH, 'wb') as pickle_file:
    pickle.dump(pipeline, pickle_file)

In [ ]:
_FILEPATH = os.path.join(os.path.dirname(os.getcwd()), 'references', 'models', 'day_2_calib_multi_prod.pkl')

with open(_FILEPATH, 'wb') as pickle_file:
    pickle.dump(calibrated_model, pickle_file)

<h4>Формирование сегмента клиентов для пилота</h4>

In [ ]:
df_prod_score = prod_features.merge(feature_matrix_score, how='right')
X_score = df_prod_score.drop(['user_id'], axis=1)

In [ ]:
scores = pipeline.predict_proba(X_score)

In [ ]:
score_df = pd.DataFrame()
score_df['customer_id'] = df_prod_score.user_id
score_df['report_date'] = dt.date.today()
score_df['score'] = scores[:,1]

In [ ]:
score_df.sort_values('score', ascending=False).iloc[:int(cutoff * score_df.shape[0]) +1, 0].values

<h3>Домашнее задание 4</h3>
<br>
Сбор витрины<br>
Добавьте информацию о пяти наиболее популярных товарах относящихся к категориям 'MEAT', 'MEAT-WHSE', 'MEAT-PCKGD' или 'PORK'.

<b><i>Подсказка:</i></b> Создайте массив со всеми product_id, относящихся к этим категориям и используйте фильтрацию c помощью метода isin().

<br>
<h1>Практика 4</h1>
<h3>Построение модели склонности для клиентов без истории</h3>

<h4>Сбор обучающей выборки</h4>

In [ ]:
# создадим еще одну сущность EntitySet, содержащую меньше признаков, например, не учитывающую предпочтения в категориях
es = ft.EntitySet("a2nced_sample")

In [ ]:
# сначала добавим созданные датафреймы
es.add_dataframe(dataframe_name='customer_info',
                 dataframe=demographics,
                 index='user_id')

es.add_dataframe(dataframe_name='orders',
                 dataframe=orders_df,
                 index='basket_id',
                 time_index='purch_date_time')

es.add_dataframe(dataframe_name='transactions',
                dataframe=transactions_df.loc[transactions_df.user_id.isin(available_ids)],
                index='record_id',
                time_index='purch_date_time')

es.add_dataframe(dataframe_name='products',
                dataframe=prod_df,
                index='product_id')

In [ ]:
# теперь укажем поля для построения связей между ними
es.add_relationship('customer_info', 'user_id', 'orders', 'user_id')
es.add_relationship('customer_info', 'user_id', 'transactions', 'user_id')
es.add_relationship('orders', 'basket_id', 'transactions', 'basket_id')
es.add_relationship('products', 'product_id', 'transactions', 'product_id')

In [ ]:
# первый заказ какого-либо товара
# используем менее насыщенный признаками датаст
df = create_feature_matrix(es, '2018-10-08')

<h4>Разметка целевого события</h4>

In [ ]:
# используем для выявления целевых событий и отсечения ложноположительных случаев операции над множествами (разность множеств)
prev_buy = set(transactions_df.loc[(transactions_df.user_id.isin(available_ids))\
                                   &(transactions_df.purch_date_time <=dt.datetime(2018, 10, 8))\
                                   &(transactions_df.product_id==1082185)]['user_id'].unique())
cur_buy = set(transactions_df.loc[(transactions_df.user_id.isin(available_ids))\
                                  &(transactions_df.purch_date_time >=dt.datetime(2018, 10, 8))\
                                  &(transactions_df.purch_date_time < dt.datetime(2018, 10, 22))\
                                  &(transactions_df.product_id==1082185)]['user_id'].unique())

In [ ]:
target = pd.DataFrame()
target['user_id'] = list(cur_buy.difference(prev_buy))
target['y'] = 1
df = df.merge(target, how='left', on='user_id')

df['y'] = df['y'].fillna(0)
df['y'] = df['y'].astype('int8')

y = df['y']
X = df.drop(columns=['y', 'user_id'], axis=1)

In [ ]:
y.value_counts()

<h3>Домашнее задание 5</h3>
<br>
Для полученной выборки разметьте таргет<br>
Целевое событие: первая покупка клиентом товара с id 1029743 в период двух недель от даты витрины.

<b><i>Подсказка:</i></b> Можно использовать предложенный подход либо определить для каждого клиента дату первой покупки указанного товара и отфильтровать ее по дате.

<h3>Подбор оптимальных гиперпараметров и обучение модели</h3>

In [ ]:
for column in X.select_dtypes(exclude=['Int64', 'float64']).columns.values:
    X[column] = X[column].astype('object')

In [ ]:
X = X.replace({'<NA>': np.nan})
X = X.replace({True: 1, False: 0})
X = X.replace({pd.NA: np.nan})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [ ]:
categorical_features = X_train.select_dtypes(exclude=['Int64', 'float64']).columns.values
numeric_features = [feature for feature in list(X.columns) if feature not in categorical_features]

In [ ]:
pipeline = create_pipeline(cat_features=categorical_features,
                          numer_features=numeric_features,
                          model=LGBMClassifier(objective = "binary", random_state = 42, is_unbalance=True))

In [ ]:
best = param_search(pipeline, X_train, y_train, search_space)

In [ ]:
best_params = unpack_model_params(best)
pipeline.set_params(**best_params)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
prob_pred = pipeline.predict_proba(X_test)

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
plot_roc_curve(pipeline, X_test, y_test)

<h4>Оценка качества модели на отложенной выборке (OOT - out of time)</h4>

In [ ]:
df_oot  = create_feature_matrix(es, '2018-10-22')

In [ ]:
prev_buy = set(transactions_df.loc[(transactions_df.user_id.isin(available_ids))\
                                   &(transactions_df.purch_date_time <=dt.datetime(2018, 10, 22))\
                                   &(transactions_df.product_id==1082185)]['user_id'].unique())

In [ ]:
cur_buy = set(transactions_df.loc[(transactions_df.user_id.isin(available_ids))\
                                  &(transactions_df.purch_date_time >=dt.datetime(2018, 10, 22))\
                                  &(transactions_df.purch_date_time < dt.datetime(2018, 11, 5))\
                                  &(transactions_df.product_id==1082185)]['user_id'].unique())

In [ ]:
target = pd.DataFrame()
target['user_id'] = list(cur_buy.difference(prev_buy))
target['y'] = 1
df_oot = df_oot.merge(target, how='left', on='user_id')

df_oot['y'] = df_oot['y'].fillna(0)
df_oot['y'] = df_oot['y'].astype('int8')

y_oot = df_oot['y']
X_oot = df_oot.drop(columns=['y', 'user_id'], axis=1)

In [ ]:
y_oot.value_counts()

In [ ]:
for column in X_oot.select_dtypes(exclude=['Int64', 'float64']).columns.values:
    X_oot[column] = X_oot[column].astype('object')
X_oot.info()

In [ ]:
X_oot = X_oot.replace({'<NA>': np.nan})
X_oot = X_oot.replace({True: 1, False: 0})
X_oot = X_oot.replace({pd.NA: np.nan})

In [ ]:
y_pred_oot = pipeline.predict(X_oot)
y_pred_prob_oot = pipeline.predict_proba(X_oot)

In [ ]:
print(f'Accuracy: {accuracy_score(y_oot, y_pred_oot)}')

In [ ]:
plot_roc_curve(pipeline, X_oot, y_oot)

In [ ]:
from utils.metrics import plot_metrics

In [ ]:
import ipywidgets as widgets
thr_hold = widgets.FloatSlider(value=0.5,
                               min=0.0,
                               max=1.0,
                               step=0.01)

In [ ]:
def plot_con(threshold):
    plot_metrics(y_oot, y_pred_prob_oot, threshold)

In [ ]:
widgets.interact(plot_con,
                 threshold=thr_hold)

In [ ]:
from utils.metrics import get_feature_names

In [ ]:
feature_names = get_feature_names(pipeline.named_steps['preprocessor'])
feature_importance = []


for i, feature in enumerate(pipeline.steps[1][1].feature_importances_):
    feature_importance.append((feature_names[i], feature))
    
    
feature_importance.sort(key=lambda x:x[1], reverse=True)
feature_importance[:10]

In [ ]:
scores = y_pred_prob_oot
scores = pd.DataFrame(scores)

scores['y'] = y_oot.values
scores_sorted = scores.sort_values(1, ascending=False)[[1, 'y']]
scores_sorted['cum_recall'] = scores_sorted['y'].cumsum()
scores_sorted['position'] = range(1, scores_sorted.shape[0]+1)
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]+1

scores.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_diff'] = scores_sorted['recall_at'].diff()
scores_sorted['position_diff'] = scores_sorted['adj_position'].diff()

scores_sorted.fillna(0, inplace=True)

scores_sorted['derivative'] = scores_sorted['position_diff']  / (scores_sorted['recall_diff']+1e-6)
scores_sorted['sum_derivative'] = scores_sorted['derivative'].cumsum()

scores_sorted.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]

In [ ]:
scores_sorted.plot(x='adj_position', y='recall_at', figsize=(12,8), title='Кумулятивный recall')

In [ ]:
cutoff = find_inflection_point(scores_sorted, 5, 4, 801)

In [ ]:
cutoff = cutoff / scores_sorted.shape[0]

In [ ]:
_FILEPATH = os.path.join(os.path.dirname(os.getcwd()), 'references', 'models', 'day_2_new_clients.pkl')

with open(_FILEPATH, 'wb') as pickle_file:
    pickle.dump(pipeline, pickle_file)

<h4>Формирование сегмента клиентов для пилота</h4>

In [ ]:
df_score = create_feature_matrix(es, '2018-11-05')

In [ ]:
X_score = df_score.drop(columns=['user_id'], axis=1)

In [ ]:
for column in X_score.select_dtypes(exclude=['Int64', 'float64']).columns.values:
    X_score[column] = X_score[column].astype('object')
X_score.info()

In [ ]:
X_score = X_score.replace({'<NA>': np.nan})
X_score = X_score.replace({True: 1, False: 0})
X_score = X_score.replace({pd.NA: np.nan})

In [ ]:
scores = pipeline.predict_proba(X_score)

In [ ]:
score_df = pd.DataFrame()
score_df['customer_id'] = df_score.user_id
score_df['report_date'] = dt.date.today()
score_df['score'] = scores[:,1]

In [ ]:
score_df.sort_values('score', ascending=False).iloc[:int(cutoff * score_df.shape[0]) +1, 0].values

<br>
<h1>Практика 5</h1>
<h3>Построение модели склонности для естественного трафика</h3>

<h4>Сбор обучающей выборки</h4>

In [ ]:
# подготовим данные
transactions_df['month'] = pd.DatetimeIndex(transactions_df['purch_date_time']).month
transactions_df['season'] = transactions_df['month'].apply(lambda x: x%12 // 3 + 1)
transactions_df['weekday'] = pd.DatetimeIndex(transactions_df['purch_date_time']).weekday
transactions_df['hour'] = pd.DatetimeIndex(transactions_df['purch_date_time']).hour
transactions_df.head()

In [ ]:
df = transactions_df.groupby(['season', 'month', 'weekday', 'hour', 'product_id'])['product_id'].count() \
/ transactions_df.groupby(['season', 'month', 'weekday', 'hour'])['product_id'].count()

In [ ]:
df = df.to_frame()
df.rename({'product_id':'share'}, axis=1, inplace=True)
df_2 = df.reset_index()
df_2.sort_values(['season', 'month', 'weekday', 'hour', 'share'],
                 ascending=[True,True,True,True,False])
df_2.sample(10)

In [ ]:
# напишем простую функцию, которая будет возвращать нам информацию о наиболее подходящих товарах
def propensity_score_sort(data:pd.DataFrame,
                          season:int=None,
                          month:int=None,
                          weekday:int=None,
                          hour:int=None) -> pd.DataFrame:
    """
    
    """
    if month is None or (month < 0 or month > 12):
        month = dt.datetime.now().month
    if season is None or (season < 0 or season > 4):
        season = month % 12 // 3 + 1
    if weekday is None or (weekday < 0 or weekday > 7):
        weekday = dt.datetime.now().weekday()
    if hour is None or (hour < 0 or hour > 12):
        hour = dt.datetime.now().hour
    
    mask = ((data.season == season)\
                &(data.month == month)\
                &(data.weekday == weekday)\
                &(data.hour == hour))
    return(data.loc[mask][['product_id', 'share']].sort_values('share', ascending=False))
    

In [ ]:
# Протестируем на параметрах по умолчанию
propensity_score_sort(df_2).head(10)

In [ ]:
# а теперь конкретизируем запрос для определенного момента времени
propensity_score_sort(df_2, 1, 1, 1, 1).head(10)

<br>
<h1>Практика 6</h1>
<h3>Гиперсегментация - построение модели склонности для целевого трафика</h3>
<br>
<h4>Сбор обучающей выборки</h4>

In [ ]:
# добавим информацию о категории, в которой совершен заказ
catigs = transactions_df.merge(prod_df[['product_id', 'department']], how='left', on='product_id')

In [ ]:
# Данные для рекламной кампании
df = demographics[['user_id', 'age_desc', 'adult_gender', 'income_desc', 'no_kids']]

In [ ]:
start_period = dt.datetime.strptime('2019-07-01', '%Y-%m-%d') + dt.timedelta(days=1)
end_period =  dt.datetime.strptime('2019-07-01', '%Y-%m-%d')  + dt.timedelta(days=14)

<h4>Разметка целевого события</h4>

In [ ]:
# почситаем, сколько человек делало покупки в категории PASTRY за две недели
target = pd.DataFrame()
target['user_id'] = catigs[(catigs.department=='PASTRY') & (catigs.purch_date_time >= start_period) & (catigs.purch_date_time <= end_period)]['user_id'].unique()
target['y'] = 1

In [ ]:
df = df.merge(target, how='left', on='user_id')
df['y'] = df['y'].fillna(0).astype('int8')
df.y.value_counts()

In [ ]:
cat_features = ['age_desc', 'adult_gender', 'income_desc', 'no_kids']

In [ ]:
pipeline = create_pipeline(cat_features=cat_features,
                          numer_features=[],
                          model=LGBMClassifier(objective = "binary", random_state = 42, is_unbalance=True))

In [ ]:
X = df.drop(columns=['y', 'user_id'], axis=1)
y = df['y']

In [ ]:
for column in X.select_dtypes(exclude=['Int64', 'float64']).columns.values:
    X[column] = X[column].astype('object')
    
X = X.replace({'<NA>': np.nan})
X = X.replace({True: 1, False: 0})
X = X.replace({pd.NA: np.nan})

<h4>Подбор оптимальных значений гиперпараметров и обучение модели</h4>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [ ]:
best = param_search(pipeline, X_train, y_train, search_space)

In [ ]:
best_params = unpack_model_params(best)
pipeline.set_params(**best_params)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
prob_pred = pipeline.predict_proba(X_test)

<h4>Оценка качества модели</h4>

In [ ]:
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

In [ ]:
plot_roc_curve(pipeline, X_test, y_test)

<h4>Оценка качества модели на отложенной выборке (ООТ - Out Of Time)</h4>

In [ ]:
start_period = dt.datetime.strptime('2019-07-15', '%Y-%m-%d') + dt.timedelta(days=1)
end_period =  dt.datetime.strptime('2019-07-15', '%Y-%m-%d')  + dt.timedelta(days=14)

In [ ]:
# почситаем, сколько человек делало покупки в категории PASTRY за две недели
target = pd.DataFrame()
target['user_id'] = catigs[(catigs.department=='PASTRY') & (catigs.purch_date_time >= start_period) & (catigs.purch_date_time <= end_period)]['user_id'].unique()
target['y'] = 1

In [ ]:
df.drop('y', axis=1, inplace=True)

df = df.merge(target, how='left', on='user_id')
df['y'] = df['y'].fillna(0).astype('int8')
df.y.value_counts()

In [ ]:
X_oot = df.drop(columns=['y', 'user_id'], axis=1)
y_oot = df['y']

In [ ]:
y_pred_oot = pipeline.predict(X_oot)
y_pred_prob_oot = pipeline.predict_proba(X_oot)

In [ ]:
print(f'Accuracy: {accuracy_score(y_oot, y_pred_oot)}')

In [ ]:
plot_roc_curve(pipeline, X_oot, y_oot)

In [ ]:
_FILEPATH = os.path.join(os.path.dirname(os.getcwd()), 'references', 'models', 'day_2_targeted.pkl')

with open(_FILEPATH, 'wb') as pickle_file:
    pickle.dump(calibrated_model, pickle_file)

<h4>Формирование сегмента клиентов для пилота</h4>

In [ ]:
scores = y_pred_prob_oot
scores = pd.DataFrame(scores)

scores['y'] = y_oot.values
scores_sorted = scores.sort_values(1, ascending=False)[[1, 'y']]
scores_sorted['cum_recall'] = scores_sorted['y'].cumsum()
scores_sorted['position'] = range(1, scores_sorted.shape[0]+1)
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]+1

scores.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_diff'] = scores_sorted['recall_at'].diff()
scores_sorted['position_diff'] = scores_sorted['adj_position'].diff()

scores_sorted.fillna(0, inplace=True)

scores_sorted['derivative'] = scores_sorted['position_diff']  / (scores_sorted['recall_diff']+1e-6)
scores_sorted['sum_derivative'] = scores_sorted['derivative'].cumsum()

scores_sorted.sort_values(1, ascending=False).head(10)

In [ ]:
scores_sorted['recall_at'] = scores_sorted['cum_recall'] / scores_sorted['y'].sum()
scores_sorted['adj_position'] = scores_sorted['position'] / scores_sorted.shape[0]

In [ ]:
scores_sorted.plot(x='adj_position', y='recall_at', figsize=(12,8), title='Кумулятивный recall')

In [ ]:
cutoff = find_inflection_point(scores_sorted, 5, 4, 801, step=25)

In [ ]:
cutoff

In [ ]:
cutoff = cutoff / scores_sorted.shape[0]

In [ ]:
X_score = df.drop(['user_id', 'y'], axis=1)

In [ ]:
scores = pipeline.predict_proba(X_score)

In [ ]:
score_df = pd.DataFrame()
score_df['customer_id'] = df.user_id
score_df['report_date'] = dt.date.today()
score_df['score'] = scores[:,1]

In [ ]:
score_df.sort_values('score', ascending=False).iloc[:int(cutoff * score_df.shape[0]) +1, 0].values